# Gradient descent from scratch - part 2, training a neural net

## Introduction
Welcome back. In part one you were introduced to the basic concepts of how gradient descent works, and used it to solve a simple regression problem. Now we are going to build on what was learnt in the last lesson and use gradient descent to solve a more difficult problem, training a neural network to classify handwritten digits. The format will be short explanations, followed by python code you can run from within this notebook.

## Image classification and MNIST
Image classification is the task of determining what category (class) an image belongs to. Today we will be performing classification on images from the MNIST dataset, which contains scanned images of handwritten digits (0-9). MNIST is a commonly used dataset in machine learning, especially for teaching and research. It contains 70,000 28x28 pixel greyscale images and corresponding labels. The labels are an integer between 0 and 9 and refer to the digit depicted in that particular image. The dataset was created by the United States' National Institute of Standards and Technology. [More information](http://yann.lecun.com/exdb/mnist/).

Ok lets load up the datset and take a look at it:

In [ ]:
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt
%matplotlib inline

with open('mnist_normalized.pkl', 'rb') as f:
    train_set, val_set, __ = pickle.load(f)

rows = 5
cols = 4

fig_1 = plt.figure(1, figsize=(10, 10))
for k in range(1, rows * cols + 1):
    index = random.randint(0, len(train_set))
    img = train_set[index][0]
    label = train_set[index][1]
    sub = fig_1.add_subplot(rows, cols, k)
    sub.axis('off')
    sub.imshow(img.reshape(28, 28), cmap='gray')
    sub.set_title(np.argmax(label))

plt.show()

As you can see from looking at the images, this would not be a trivial problem to solve if you had to invent a solution from scratch. Luckily neural networks and gradient descent have been invented, so we should not have too much trouble.

## Artificial Neurons
In part one, we were able to use a very simple linear model (*y = mx + b*), however the problem we are trying to solve here is quite a bit more complex. Neural networks are a type of machine learning model that are good at this sort of problem. 

The basic unit of a neural network is the artificial neuron. The type we will be using are called sigmoid neurons. A sigmoid neuron takes *n* inputs that are each multiplied by an associated weight variable. The sum of all these weighted inputs is then added to added to a bias value unique to that neuron. Then the sigmoid fucntion is applied to get the output of the neuron (often refered to as the *activation* of the neuron). 

![sigmoid neuron](res/ax7.jpg)

Below is a more formal mathematical definition.

$ a = \sigma ((\sum _n x_n*w_n) + b)$

Where $a$ is the output of the neuron, $x_n$ refers to the each input to the neuron, $w_n$ refers to the weight variable associated with each input, and $b$ is the bias value for the neuron.

## Neural networks

The most simple neural networks are just a collection of neurons arranged in layers. For our image classification problem we are going to have a network comprised of three layers of sigmoid neurons. The first layer will be the input image (flattened into a 784, vector). The second layer will consist of 32 neurons, and the final layer will consist of 10 neurons. Each neuron in the final layer will represent a possible prediction of the network (digits 0 - 9). The neuron in the final layer with the highest activation will be the prediction of the network. Each neuron in the first layer will connect to each neuron in the second layer and each neuron in the second layer will connect to each neuron in the third layer. Each of these connections will have a unique weight variable associated with it. Each neuron in the second and thrid layers will have a unique bias variable. This network archirtechture is picture below.

![network achitechture](res/ax8.jpg)

Using matrix notation We can write an entire layer of the neural network as follows:

$ a^l = \sigma(w^l a^{l-1} + b^l) $

Where $a^l$ is a vector containing all the outputs of the neurons in layer $l$, $w$ is a matrix containing all the weights connecting this layer with the previous layer, $a^{l-1}$ is a vector containing all the inputs to this layer and $b$ is a vector containing the bias values for each neuron in this layer.

Expressing our networks in this form is much less cumbersome and is easier to work with.

As you can imagine, the functionality of a neural network can be very different based on the weight and bias values. I wonder if there is a way to optimize these weight and bias values to get our network to output correct predictions from an input MNIST image? Hint: It begins with 'g' and ends with 'radient descent' ....

## Training our neural network with gradient descent

So what we want to do is find values for all the weight and bias variables so that our network makes correct predictions.

We will follow the same overall strategy as we did in part one.

1. Start with random values for the variables we are trying to optimize
2. Compute some form of error measurement for our predictive model using know input/output pairs
3. Use this error measurement to calculate a gradient for each variable
4. Change each variable based on its gradient
5. Repeat steps 2-4 until variables are (hopefully) optimized

The difference here is that we are trying to optimize 25,450 variables instead of just two.

First we need a cost funtion. Here we are using quadratic cost:

$ C(w, b) = \frac {1}{2} (y - a)^2 $

Where $w$ and $b$ refers to the weight and bias variables we are trying to optimize, a is our models prediction and y is the ground truth from our dataset.

So your probably wondering how we are going to calculate all these partial derivatives?

We could try and compute all the partial derivatives indiviually like in part one, but with so many trainable variables this would get $very$ messy very quickly. Luckily there is an algoithm called backpropagation that allows us to calculate these gradients in a less cumbersome way.

## Backpropagation

Backpropogation is an algorithm that allows alot of gradients to be computed in an efficient manner.

Before we can get to the details we need to define / clarify  a few things.

$ w^l_{jk} $ weight connecting $k^{th}$ neuron in the $l-1^{th}$ layer to the $j^{th}$ neuron in the $l^{th}$ layer.

$ b^l_j $ bias for the $j^{th}$ neuron in the $l^{th}$ layer.

$ a^l_j $ activation (output) of the $j^{th}$ neuron in the $l^{th}$ layer

$ z^l $ weighted inputs to neurons in layer $l$ (after weights and bias, before sigmoid function)

$ \delta^l_j $ error in the $j^{th}$ neuron in the $l^{th}$ layer (how the cost changes with respect to changes in the weighted input to neurons in layer $l$)

Recall from the previous section that we can write an entire layer of the neural network as follows:

$ a^l = \sigma(w^l a^{l-1} + b^l) $

or another way of expressing it:

$ z^l \equiv w^l a^{l-1} + b^l $

$ a^l = \sigma(z^l) $

So now lets get to backpropagation. In the context of our problem it has four main steps.

1. Calculate the $\delta$ for the final layer of the network ($\delta^L$)

2. Calculate $\delta$ for the other layers in the network, working backwards from the final layer

3. Calculate the partial derivative for each weights matrix

4. Calculate the partial derivative for each bias vector

Each of these steps can be solved by the respective equation below. $\bigodot$ refers to the [hadamard product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices). If you are interested in the proofs for these equations, check out this website <a href="http://neuralnetworksanddeeplearning.com/chap2.html#proof_of_the_four_fundamental_equations_(optional)"> here </a>. It is a great resource if you want to dive deeper into the maths behind neural networks and deep learning. We will name each of the equations BP1 - 4 so that we can refer to them in the code below. Note that these backpropogation equations will differ with different cost functions and network architechtures, but the same concepts apply.

BP1: $\delta^L = (a^L - y) \bigodot \sigma'(z^L) $
    
BP2: $\delta^l = ((w^{l+1})^T \delta^{l+1}) \bigodot \sigma'(z^l) $

BP3: $\frac {\partial C}{\partial b^l_j} = \delta^l_j $

BP4: $\frac {\partial C}{\partial w^l_{jk} } = a^{l-1}_k \delta^l_j $

If at any point in the code the math is not making sense, refer back to this section.


## Mini-batch gradient descent

In part one we trained the network on one data point at a time. The disadvantage of this is that one anomalous data point can update your trainable variables in a disadventageous way. Mini batch gradient descent involves calculating the gradients for multiple data points, then updating the trainable variables by the average of there gradeints, thus reducing the impact of extreme outliers.

## Train, validation and test sets

In machine learning it is common to split you data into two or three sets. The training set is the data that you use to train your machine learning model. The validation set is the data that you use to test your machine learning model as you train it. The reason you don't test on the training set is you want to know if your model has actually learned to predict new examples it has not seen before (if you test it on the training set it may have just memorized the training set and may not be able to predict new examples). In a production setting it is common to have a thrid dataset to test your final system on before you release it. For our tutorial today two datasets (train and validation) will suffice.

Ok, now we are ready to implement our neural network in code.

Note that the MNIST dataset that we loaded has already been normalized so we don't need to do that ourselves.

First we will create any helper functions that we need. The purpose is explained in the comments.

In [ ]:
def mnist_data_generator(dset, batch_size):
    """ generator function that yields batches of (image, label) pairs """
   
    assert 0 < batch_size <= len(dset)

    random.shuffle(dset)
    index = 0

    while True:
        yield dset[index : index + batch_size]
        index += batch_size
        if index + batch_size > len(dset):
            random.shuffle(dset)
            index = 0

Now we will create our network class. It is simply applying what we have discussed up to this point. The comments should help you understand what each part of the code does.

In [ ]:
""" neural network class. takes as argument a list of integers, refering to the number of neurons in each layer """ 
class Net():

    def __init__(self, layers):
        self.layers = layers
        self.num_layers = len(layers)
        self.weights = [np.random.randn(y, x) for y, x in zip(layers[1:], layers[:-1])] # list of network weight variables
        self.biases = [np.random.randn(y, 1) for y in layers[1:]] # list of network bias variables

    @staticmethod
    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    @staticmethod
    def sigmoid_prime(z):
        return Net.sigmoid(z) * (1 - Net.sigmoid(z))

    def test_batch(self, batch):
        """ returns the number of images in batch correctly classified by the network """
        return sum( (1 if np.argmax(self.forward_prop(x)) == np.argmax(y) else 0 for x, y in batch) )
        
    def forward_prop(self, a):
        """ calculates output of network for a given input image"""
        for w, b in zip(self.weights, self.biases):
            a = self.sigmoid(np.dot(w, a) + b)
        return a

    def calc_gradients(self, x, y):
        """ calculates gradients for weights and biases on a single image/label pair """
        
        w_gradients = [np.zeros(w.shape) for w in self.weights] # gradients for all weight variables
        b_gradients = [np.zeros(b.shape) for b in self.biases] # gradients for all bias variables
        activations = [] # list containing output of all layers
        weighted_inputs = [] # list containing z value for all layers (pre activation weighted inputs)

        activation = x  # set initial 'activation' to input
        activations.append(activation)

        # forward_prop
        for w, b in zip(self.weights, self.biases):
            weighted_input = np.dot(w, activation) + b # z = w * x + b, where x in output of previous layer
            weighted_inputs.append(weighted_input)
            activation = self.sigmoid(weighted_input) # a = sigmoid(z)
            activations.append(activation)

        # backward prop
        # start with final layer
        delta = (activations[-1] - y) * self.sigmoid_prime(weighted_inputs[-1]) # BP1
        w_gradients[-1] = np.dot(delta, activations[-2].transpose()) # BP4
        b_gradients[-1] = delta # BP3

        # for each layer (starting 2nd to last & moving back) 
        for l in range(-2, self.num_layers * -1, -1): 
            delta = np.dot(self.weights[l + 1].transpose(), delta) * self.sigmoid_prime(weighted_inputs[l]) # BP2
            w_gradients[l] = np.dot(delta, activations[l-1].transpose()) # BP4
            b_gradients[l] = delta # BP3

        return (w_gradients, b_gradients)

    def train_batch(self, batch, learning_rate):
        """ a single training iteration on a mini batch """
        
        w_gradients = [np.zeros(w.shape) for w in self.weights] # will contain gradients for all weight variables
        b_gradients = [np.zeros(b.shape) for b in self.biases] # will contain gradients for all bias variables
        
        # calculate gradients for entire batch
        for img, label in batch:
            delta_w_gradients, delta_b_gradients = self.calc_gradients(img, label.reshape(-1, 1))
            w_gradients = [nw + dnw for nw, dnw in zip(w_gradients, delta_w_gradients)]
            b_gradients = [nb + dnb for nb, dnb in zip(b_gradients, delta_b_gradients)]
        
        # take average gradient over batch
        w_gradients = [nw / len(batch) for nw in w_gradients]
        b_gradients = [nb / len(batch) for nb in b_gradients]

        # update weights and biases
        self.weights = [w - nw * learning_rate for w, nw in zip(self.weights, w_gradients)]
        self.biases = [b - nb * learning_rate for b, nb in zip(self.biases, b_gradients)]

    def train(self, train_set, val_set, batch_size, learning_rate, n_iterations, n_test):
        """ main method for controlling training process """
        
        # setup generators
        train_gen = mnist_data_generator(dset=train_set, batch_size=batch_size)
        val_gen = mnist_data_generator(dset=val_set, batch_size=n_test)
        
        # train for n_iterations
        for i in range(1, n_iterations + 1, 1):
            
            self.train_batch(batch=next(train_gen), learning_rate=learning_rate)
            
            if i % 1000 == 0: # print out update of training process
                print('Iteration {0}: val_acc = {1}/{2}'.format(i, self.test_batch(next(val_gen)), n_test) )


Now lets instantiate and train our neural net. Be patient, it will take a couple of minutes. An update to the training process will be printed every 1000 iterations.

In [ ]:
net_1 = Net(layers=[784, 32, 10]) # instantiate network class
net_1.train(train_set, val_set, batch_size=20, learning_rate=3.0, n_iterations=25000, n_test=1000) # train network

In [ ]:
# plot 20 random images from validation set alongside our networks prediction

rows = 5
cols = 4

fig_2 = plt.figure(1, figsize=(10, 10))
for k in range(1, rows * cols + 1):
    index = random.randint(0, len(val_set))
    img = val_set[index][0]
    label = np.argmax(val_set[index][1])
    pred = np.argmax(net_1.forward_prop(img))
    sub = fig_2.add_subplot(rows, cols, k)
    sub.axis('off')
    sub.imshow(img.reshape(28, 28), cmap='gray')
    clr = 'green' if label == pred else 'red'
    sub.set_title('pred: {}'.format(pred), color=clr)

plt.show()

# print networks performance on entire validation set
print('Validation set performance: {0}/{1}'.format(net_1.test_batch(val_set), len(val_set) ) )

Well done. You should have obtained a classification accuracy between 93 and 96% on the validation set. If you got significantly less than this, try running the model again (random initialization of weights and random shuffling of training data mean every training run will be different).

This is good performance for something we hacked together in python without the use of any machine learning libraries. State of the art performance for MNIST is >99.7% classification accuracy, so we have some room for improvement. There are many ways to improve performance, but by far the best way would be to use a convolutional neural net (CNN). Our network here flattened the images into a (784,) vector, which threw away alot of the useful information. A CNN would take into account the spatial relashionship between the pixels.

Hopefully you have learnt a bit about the inner workings of gradient descent and machine learning in general. I also hope you got a new found appreciation for machine learning libraries that allow us to focus on building machine learning systems rather than implementing backpropogation by hand every time!